In [2]:
import pandas as pd
import re
from difflib import SequenceMatcher
import textwrap

In [3]:
# -----------------------------
# Load data from CSV
# -----------------------------
df = pd.read_csv("xai_prompt_sensitivity_demo.csv")

In [4]:
# -----------------------------
# Helper functions
# -----------------------------
def wrap(text, width=90):
    return "\n".join(textwrap.wrap(str(text), width))

def normalize(text):
    return re.sub(r"\s+", " ", str(text)).strip().lower()

def similarity(a, b):
    return SequenceMatcher(None, normalize(a), normalize(b)).ratio()

def extract_signals(text):
    text = normalize(text)
    confidence_terms = ["definitely", "certainly", "will", "best", "perfect"]
    caution_terms = ["may", "might", "can", "could", "consider"]
    compliance_terms = ["not investment advice", "review risks", "disclosures"]

    return {
        "confidence_terms": sum(text.count(t) for t in confidence_terms),
        "caution_terms": sum(text.count(t) for t in caution_terms),
        "compliance_terms": sum(text.count(t) for t in compliance_terms),
        "word_count": len(text.split())
    }

In [5]:
# -----------------------------
# Demo Part 1: Prompt sensitivity
# -----------------------------
print("\n=== PROMPT SENSITIVITY DEMO ===\n")

p1 = df[df["pair_id"] == "P1"].reset_index(drop=True)

for _, row in p1.iterrows():
    print(f"{row['case_id']} | {row['label']}")
    print("Prompt:")
    print(wrap(row["prompt"]))
    print("\nOutput:")
    print(wrap(row["output"]))
    print("\n" + "-" * 90 + "\n")

print("Similarity score:",
      similarity(p1.loc[0, "output"], p1.loc[1, "output"]))


=== PROMPT SENSITIVITY DEMO ===

P1-A | Baseline prompt
Prompt:
Write a client-facing product description for 'ClearNest Balanced Portfolio'. Audience:
moderate-risk investors. Tone: professional, cautious, clear. Include a brief note about
market conditions and risk. 120-160 words.

Output:
ClearNest Balanced Portfolio is designed for moderate-risk investors seeking a diversified
approach across equities and high-quality fixed income. In today’s market, diversification
can help manage volatility while maintaining exposure to long-term growth opportunities.
This portfolio emphasizes balance, aiming to reduce overreliance on any single asset
class. As with any investment, risk is present and market conditions can change. Consider
how this strategy aligns with your time horizon and objectives, and review key risks
before making decisions.

------------------------------------------------------------------------------------------

P1-B | Small prompt change (add 'high-performing')
Prompt

In [6]:
# -----------------------------
# Demo Part 2: Counterfactual pair
# -----------------------------
print("\n=== COUNTERFACTUAL EVALUATION ===\n")

p2 = df[df["pair_id"] == "P2"].reset_index(drop=True)

for _, row in p2.iterrows():
    print(f"{row['case_id']} | {row['label']}")
    print("Output:")
    print(wrap(row["output"]))
    print("\n" + "-" * 90 + "\n")

print("Similarity score:",
      similarity(p2.loc[0, "output"], p2.loc[1, "output"]))


=== COUNTERFACTUAL EVALUATION ===

P2-A | Counterfactual A (Brand: ClearNest)
Output:
ClearNest Balanced Portfolio offers a measured approach for moderate-risk investors,
combining diversified equity exposure with stabilizing fixed income. In the current
environment, this balance can help manage volatility while maintaining participation in
long-term trends. The strategy is positioned as a core holding for investors who value
discipline and diversification. Review risks and disclosures, and consider personal
objectives before investing.

------------------------------------------------------------------------------------------

P2-B | Counterfactual B (Brand: PrimeStone)
Output:
PrimeStone Balanced Portfolio is an attractive choice for moderate-risk investors, pairing
diversified equity exposure with high-quality fixed income to support growth and
stability. With improving sentiment in several market segments, this portfolio is
positioned to capture upside while maintaining balance. I

In [7]:
# -----------------------------
# Demo Part 3: Unexpected behavior
# -----------------------------
print("\n=== UNEXPECTED BEHAVIOR ===\n")

p3 = df[df["pair_id"] == "P3"].reset_index(drop=True)

for _, row in p3.iterrows():
    print(f"{row['case_id']} | {row['label']}")
    print("Output:")
    print(wrap(row["output"]))
    print("\n" + "-" * 90 + "\n")


=== UNEXPECTED BEHAVIOR ===

P3-A | Unexpected behavior A (neutral compliance reminder)
Output:
ClearNest Balanced Portfolio provides diversified exposure across equities and fixed
income for moderate-risk investors. The strategy aims to balance growth potential with
risk management in changing market conditions. This description is for informational
purposes and is not investment advice. Review risks and disclosures before making
decisions.

------------------------------------------------------------------------------------------

P3-B | Unexpected behavior B (engaging tone shift)
Output:
ClearNest Balanced Portfolio is the perfect way to capture market opportunity while
staying balanced. With a smart mix of equities and fixed income, it helps you pursue
growth without losing sight of stability. Results may vary and this is not investment
advice. Review risks and disclosures before investing!

------------------------------------------------------------------------------------------

In [8]:
# -----------------------------
# Signal comparison table
# -----------------------------
print("\n=== SIGNAL COMPARISON ===\n")

signals = df["output"].apply(extract_signals).apply(pd.Series)
signal_df = pd.concat([df[["pair_id", "case_id", "label"]], signals], axis=1)

signal_df.sort_values(by=["pair_id", "case_id"])


=== SIGNAL COMPARISON ===



,pair_id,case_id,label,confidence_terms,caution_terms,compliance_terms,word_count
0,P1,P1-A,Baseline prompt,0,3,0,76
1,P1,P1-B,Small prompt change (add 'high-performing'),0,3,1,74
2,P2,P2-A,Counterfactual A (Brand: ClearNest),0,2,2,59
3,P2,P2-B,Counterfactual B (Brand: PrimeStone),0,1,0,63
4,P3,P3-A,Unexpected behavior A (neutral compliance remi...,0,0,3,46
5,P3,P3-B,Unexpected behavior B (engaging tone shift),1,1,3,48
